In [1]:
import amostra
import pandas
import amostra.client.commands as acc
import collections

In [2]:
# Definitions
owner = "hhslepicka"
project = "12345"
beamline_id = "lix"

In [3]:
container_ref = acc.ContainerReference()
sample_ref = acc.SampleReference()

In [4]:
def insert_plate(plate_info):
    plate = container_ref.create(**plate_info)
    return plate

def insert_sample_list(samples):
    result = sample_ref.create_sample_list(samples)
    return result

In [5]:
def generate_plate(owner, project, beamline_id, kind, name, barcode, content):
    plate_info = {    
            "owner": owner,
            "project": project,
            "beamline_id": beamline_id,
            "kind": kind,
            "name": name,
            "barcode": barcode,
            "content": list(content)
    }

    return insert_plate(plate_info)

def generate_samples_96wp(project, beamline_id, owner, name):
    samples = collections.deque()
    for i in range(96):
        sample_info = {
            "project": project,
            "beamline_id": beamline_id,
            "owner": owner,
            "name": name+" {}".format(i+1),
            "position": {"x": int(i % 8), "y": int(i/8)},
            "concentration": 0.001,
            "volume": 10,
            "temperature": 21.4    
        }
        samples.append(sample_info)
    uids = insert_sample_list(samples)
    return uids

def find_plate_by_barcode(owner, project, beamline_id, barcode):
    samples = collections.deque()
    plate_info = list(container_ref.find(owner=owner, project=project, beamline_id=beamline_id, barcode=barcode))[0]
    for s_uid in plate_info['content']:
        samples.append(next(sample_ref.find(uid=s_uid)))
    
    plate_info['content'] = list(samples)
    return plate_info

In [6]:
number_of_plates = 1

In [ ]:
for i in range(number_of_plates):
    samples = generate_samples_96wp(project, beamline_id, owner, "Test Sample")
    plate1 = generate_plate(owner, project, beamline_id, "96wp", "Plate {}".format(i+1), "{}".format(i+1).zfill(13), samples)

In [7]:
def import_plate_from_excel(fname, owner, project, beamline_id, plate_kind):
    samples = collections.deque()
    excel_data = pandas.read_excel(fname,header=1)
    
    for line in excel_data.iterrows():
        if line[0] == 0:
            name = line[1][0]
            barcode = str(int(line[1][1])).zfill(13)
            plate_info = {    
                "owner": owner,
                "project": project,
                "beamline_id": beamline_id,
                "kind": plate_kind,
                "name": name,
                "barcode": barcode,
            }
        s_x = line[1][2]
        s_y = line[1][3]
        s_name = line[1][4]
        s_shortname = line[1][5]
        s_conc = line[1][6]
        s_volume = line[1][7]
        s_temperature = line[1][8]
        sample_info = {
                "project": project,
                "beamline_id": beamline_id,
                "owner": owner,
                "name": s_name,
                "short_name": s_shortname,
                "position": {"x": s_x, "y": s_y},
                "concentration": s_conc,
                "volume": s_volume,
                "temperature": s_temperature        
            }
        samples.append(sample_info)
        
    return plate_info, samples

In [8]:
excel_file = "Sample_Import.xlsx"
plate, samples = import_plate_from_excel(excel_file, owner, project, beamline_id, "96wp")

In [9]:
content = insert_sample_list(samples)
plate['content'] = content
inserted_plate = insert_plate(plate)

In [10]:
inserted_plate

'ae25ad97-9122-4380-8949-045d5c01c9ab'

In [11]:
r = find_plate_by_barcode(owner, project, beamline_id, "1000000000001".zfill(13))

In [12]:
r["name"]

'Imported Plate'

In [13]:
for sample in r["content"]:
    print("Sample Data: ", sample)

Sample Data:  {'position': {'y': 1, 'x': 1}, 'concentration': 0.775, 'volume': 10, 'time': 1461611493.341079, 'uid': '5384ad0d-ea27-4db0-a196-38086ac3e27e', 'short_name': 'S1-1', 'name': 'Sample Imp 1-1', 'owner': 'hhslepicka', 'project': '12345', 'beamline_id': 'lix', 'temperature': 21}
Sample Data:  {'position': {'y': 2, 'x': 1}, 'concentration': 0.772, 'volume': 10, 'time': 1461611493.493785, 'uid': 'd033a02f-7dd6-4993-b1ac-944aa1044408', 'short_name': 'S1-2', 'name': 'Sample Imp 1-2', 'owner': 'hhslepicka', 'project': '12345', 'beamline_id': 'lix', 'temperature': 21}
Sample Data:  {'position': {'y': 3, 'x': 1}, 'concentration': 0.776, 'volume': 10, 'time': 1461611493.499471, 'uid': 'a50ab58e-7958-406a-a8e7-dd7e0290be65', 'short_name': 'S1-3', 'name': 'Sample Imp 1-3', 'owner': 'hhslepicka', 'project': '12345', 'beamline_id': 'lix', 'temperature': 21}
Sample Data:  {'position': {'y': 4, 'x': 1}, 'concentration': 0.47500000000000003, 'volume': 10, 'time': 1461611493.505143, 'uid': '2